In [15]:
import sys
from pathlib import Path

# Detect project root (whether you're in notebooks/ or root)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
SRC_PATH = PROJECT_ROOT / "src"

# Add src/ to sys.path if not already there
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

print("✅ src added to path:", SRC_PATH.exists())


✅ src added to path: True


In [2]:
import sys
sys.path.append("..")  # Go up one level to access src folder

from src.experiment import solve_bond_selection
print("✅ Imported successfully")


▶ src.experiment imported and running
✅ Imported successfully


In [1]:
import os, sys, pathlib

# 1) Set working directory to your repo root (where README and src/ live)
repo_root = pathlib.Path.cwd()
print("Working in:", repo_root)

# 2) Ensure Python can import from src/
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))


Working in: C:\Users\Admin\Documents\WISER_Optimization_VG\notebooks


In [3]:
%cd C:\Users\Admin\Documents\WISER_Optimization_VG


C:\Users\Admin\Documents\WISER_Optimization_VG


In [12]:
import sys
from pathlib import Path

# Detect project root (whether you're in notebooks/ or root)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
SRC_PATH = PROJECT_ROOT / "src"

# Add src/ to sys.path if not already there
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

print("✅ src added to path:", SRC_PATH.exists())


✅ src added to path: True


In [14]:
sys.path.append(str(PROJECT_ROOT))
from src.experiment import solve_bond_selection




▶ src.experiment imported and running


In [4]:
import os, sys, pathlib

repo_root = pathlib.Path.cwd()
print("Working in:", repo_root)

if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))


Working in: C:\Users\Admin\Documents\WISER_Optimization_VG


In [5]:
import os, sys, pathlib

repo_root = pathlib.Path.cwd()
print("Working in:", repo_root)

if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))


Working in: C:\Users\Admin\Documents\WISER_Optimization_VG


In [21]:
from pathlib import Path
import os

# Get current folder
cwd = Path.cwd()
print("Jupyter is running in:", cwd)

# Go up one level if you're inside /notebooks
if cwd.name == "notebooks":
    os.chdir(cwd.parent)
    print("Switched to:", Path.cwd())


Jupyter is running in: C:\Users\Admin\Documents\WISER_Optimization_VG\notebooks
Switched to: C:\Users\Admin\Documents\WISER_Optimization_VG


In [22]:
import sys
PROJECT_ROOT = Path.cwd()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("Root added to sys.path:", str(PROJECT_ROOT))


Root added to sys.path: C:\Users\Admin\Documents\WISER_Optimization_VG


In [23]:
def touch_init(path):
    path.mkdir(parents=True, exist_ok=True)
    init = path / "__init__.py"
    if not init.exists():
        init.touch()
        print("✅ Created __init__.py in", path)
    else:
        print("✔️ __init__.py already exists in", path)

# Apply to src/ and any nested package folders
for rel in ["src", "src/sbo", "src/sbo/src", "src/sbo/src/optimizer"]:
    folder = PROJECT_ROOT / rel
    if folder.exists():
        touch_init(folder)


✔️ __init__.py already exists in C:\Users\Admin\Documents\WISER_Optimization_VG\src
✅ Created __init__.py in C:\Users\Admin\Documents\WISER_Optimization_VG\src\sbo
✔️ __init__.py already exists in C:\Users\Admin\Documents\WISER_Optimization_VG\src\sbo\src
✔️ __init__.py already exists in C:\Users\Admin\Documents\WISER_Optimization_VG\src\sbo\src\optimizer


In [24]:
exp_path = PROJECT_ROOT / "src" / "experiment.py"
if not exp_path.exists():
    raise FileNotFoundError("Missing: src/experiment.py")

text = exp_path.read_text(encoding="utf-8")
replacements = {
    "from .sbo.src.optimizer.optimization_monitor": "from src.sbo.src.optimizer.optimization_monitor",
    "from .sbo.src.optimizer.optimization_wrapper": "from src.sbo.src.optimizer.optimization_wrapper",
}

patched = text
for old, new in replacements.items():
    if old in patched:
        patched = patched.replace(old, new)

if patched != text:
    backup = exp_path.with_suffix(".py.bak")
    backup.write_text(text, encoding="utf-8")
    exp_path.write_text(patched, encoding="utf-8")
    print("✅ Patched experiment.py (backup saved as experiment.py.bak)")
else:
    print("✔️ No changes needed — looks clean.")


✅ Patched experiment.py (backup saved as experiment.py.bak)


In [25]:
from src.experiment import solve_bond_selection
print("✅ Import OK")


✅ Import OK


In [1]:
LP_PATH = "data/1/31bonds/docplex-bin-avgonly.lp"  # you used this file earlier


In [20]:
LP_REL = "data/1/31bonds/docplex-bin-avgonly.lp"
x, fx, dt = solve_bond_selection(LP_REL)
print("🔢 Bonds selected:", int(np.sum(x)))
print("📈 Objective value:", round(fx, 4))
print("⏱ Runtime (seconds):", round(dt, 2))


🔢 Bonds selected: 21
📈 Objective value: 40.2771
⏱ Runtime (seconds): 166.74


In [3]:
import os
print(os.getcwd())


C:\Users\Admin\Documents\WISER_Optimization_VG\notebooks


In [4]:
import sys
sys.path.append("..")  # Moves up one level to access src/

from src.experiment import solve_bond_selection
print("✅ Imported successfully")


▶ src.experiment imported and running
✅ Imported successfully


In [3]:
import numpy as np

LP_PATH = "data/1/31bonds/docplex-bin-avgonly.lp"  # Update if needed

x, fx, dt = solve_bond_selection(LP_PATH)

print("🔢 Bonds selected:", int(np.sum(x)))
print("📈 Objective value:", round(fx, 4))
print("⏱ Runtime (seconds):", round(dt, 2))


🔢 Bonds selected: 21
📈 Objective value: 40.2771
⏱ Runtime (seconds): 181.76


In [6]:
import os

for root, dirs, files in os.walk(".."):
    for name in files:
        if name.endswith(".lp"):
            print(os.path.join(root, name))


..\data\1\31bonds\docplex-bin-avgonly-nocplexvars.lp
..\data\1\31bonds\docplex-bin-avgonly.lp
..\data\1\31bonds\docplex-bin-minmaxavg.lp
..\data\1\31bonds\docplex.lp
..\data\1\full\docplex-full-reduced-nosemi.lp
..\data\1\full\docplex-orig.lp
..\data\1\gurobi\gurobi.lp


In [1]:
LP_PATH = "data/1/31bonds/docplex-bin-avgonly.lp"



In [ ]:
x, fx, dt = solve_bond_selection(LP_PATH)

print("🔢 Bonds selected:", int(np.sum(x)))
print("📈 Objective value:", round(fx, 4))
print("⏱ Runtime (seconds):", round(dt, 2))


In [10]:
LP_PATH = "data/1/31bonds/docplex-bin-avgonly.lp"


In [13]:
x, fx, dt = solve_bond_selection(LP_PATH)

print("🔢 Bonds selected:", int(np.sum(x)))
print("📈 Objective value:", round(fx, 4))
print("⏱ Runtime (seconds):", round(dt, 2))


🔢 Bonds selected: 21
📈 Objective value: 40.2771
⏱ Runtime (seconds): 139.88


In [10]:
import pandas as pd

# Create results_df if it doesn't exist yet
if "results_df" not in globals():
    results_df = pd.DataFrame(columns=[
        "LP file", "Bonds selected", "Objective value", "Runtime (s)", "Solver", "Strategy"
    ])

# Add the tuned row
tuned_row = {
    "LP file": LP_REL,
    "Bonds selected": int(np.sum(x2)),
    "Objective value": round(fx2, 4),
    "Runtime (s)": round(dt2, 2),
    "Solver": "python-mip",
    "Strategy": "docplex-bin-avgonly (20s)"
}

results_df = pd.concat([results_df, pd.DataFrame([tuned_row])], ignore_index=True)
results_df



C:\Users\Admin\AppData\Local\Temp\ipykernel_6284\1474845570.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([tuned_row])], ignore_index=True)


,LP file,Bonds selected,Objective value,Runtime (s),Solver,Strategy
0,data/1/31bonds/docplex-bin-avgonly.lp,19,40.3562,151.43,python-mip,docplex-bin-avgonly (20s)


In [9]:
# Step 1: Set LP file path (relative to project root)
LP_REL = "data/1/31bonds/docplex-bin-avgonly.lp"

# Sanity check the path
print("LP exists?", (PROJECT_ROOT / LP_REL).exists(), "->", (PROJECT_ROOT / LP_REL))

# Step 2: Run the solver (20s time limit)
x2, fx2, dt2 = solve_mip_tuned(LP_REL, max_seconds=20, mip_gap=0.0)

# Step 3: Print results
print("🔢 Bonds selected:", int(np.sum(x2)))
print("📈 Objective value:", round(fx2, 4))
print("⏱ Runtime (seconds):", round(dt2, 2))



LP exists? True -> C:\Users\Admin\Documents\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly.lp
🔢 Bonds selected: 19
📈 Objective value: 40.3562
⏱ Runtime (seconds): 151.43


In [8]:
# Imports
import os, sys, time
from pathlib import Path
import numpy as np

# If python-mip isn't imported yet
from mip import Model

# Detect project root (works whether your notebook is in repo root or in notebooks/)
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
elif not (PROJECT_ROOT / "data").exists() and (PROJECT_ROOT.parent / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

print("PROJECT_ROOT ->", PROJECT_ROOT)

def solve_mip_tuned(lp_rel_path, *, max_seconds=20, mip_gap=0.0, threads=0):
    """
    lp_rel_path: path relative to project root, e.g., 'data/1/31bonds/docplex-bin-avgonly.lp'
    max_seconds: time limit for the MIP solve
    mip_gap: allowed relative optimality gap (not all solvers honor this via API)
    threads: number of threads (0 lets solver decide)
    """
    lp_path = (PROJECT_ROOT / lp_rel_path).resolve()
    if not lp_path.exists():
        raise FileNotFoundError(f"LP file not found at: {lp_path}")

    start = time.perf_counter()
    m = Model()
    m.read(str(lp_path))

    # Optional knobs (wrapped in try/except to avoid solver-specific attribute issues)
    try:
        if threads:
            m.threads = threads
    except Exception:
        pass

    # Use the built-in time limit on optimize
    m.optimize(max_seconds=max_seconds)

    # Collect solution
    x = np.array([v.x for v in m.vars])
    fx = float(m.objective_value)
    dt = time.perf_counter() - start
    return x, fx, dt

print("✅ solve_mip_tuned is defined")


PROJECT_ROOT -> C:\Users\Admin\Documents\WISER_Optimization_VG
✅ solve_mip_tuned is defined


In [26]:
LP_REL = "data/1/31bonds/docplex-bin-avgonly.lp"


In [27]:
x2, fx2, dt2 = solve_mip_tuned(LP_REL, max_seconds=20, mip_gap=0.0)


In [28]:
print("🔢 Bonds selected:", int(np.sum(x2)))
print("📈 Objective value:", round(fx2, 4))
print("⏱ Runtime (seconds):", round(dt2, 2))


🔢 Bonds selected: 19
📈 Objective value: 40.3562
⏱ Runtime (seconds): 157.28


In [29]:
{
    "LP file": LP_REL,
    "Bonds selected": ...,
    "Objective value": ...,
    "Runtime (s)": ...,
    "Solver": "python-mip",
    "Strategy": "tuned (20s limit)"
}


{'LP file': 'data/1/31bonds/docplex-bin-avgonly.lp',
 'Bonds selected': Ellipsis,
 'Objective value': Ellipsis,
 'Runtime (s)': Ellipsis,
 'Solver': 'python-mip',
 'Strategy': 'tuned (20s limit)'}

In [30]:
tuned_results = {
    "LP file": LP_REL,
    "Bonds selected": int(np.sum(x2)),
    "Objective value": round(fx2, 4),
    "Runtime (s)": round(dt2, 2),
    "Solver": "python-mip",
    "Strategy": "tuned (20s limit)"
}


In [36]:
results_df = pd.DataFrame([tuned_results])
results_df


,LP file,Bonds selected,Objective value,Runtime (s),Solver,Strategy
0,data/1/31bonds/docplex-bin-avgonly.lp,19,40.3562,157.28,python-mip,tuned (20s limit)


In [33]:
import numpy as np
import pandas as pd

LP_REL = "data/1/31bonds/docplex-bin-avgonly.lp"


In [14]:
baseline_results = {
    "LP file": LP_PATH,
    "Bonds selected": int(np.sum(x)),
    "Objective value": round(fx, 4),
    "Runtime (s)": round(dt, 2),
    "Solver": "python-mip",
    "Strategy": "docplex-bin-avgonly"
}

baseline_results


{'LP file': 'data/1/31bonds/docplex-bin-avgonly.lp',
 'Bonds selected': 21,
 'Objective value': 40.2771,
 'Runtime (s)': 139.88,
 'Solver': 'python-mip',
 'Strategy': 'docplex-bin-avgonly'}

In [15]:
import pandas as pd

results_df = pd.DataFrame([baseline_results])
results_df


,LP file,Bonds selected,Objective value,Runtime (s),Solver,Strategy
0,data/1/31bonds/docplex-bin-avgonly.lp,21,40.2771,139.88,python-mip,docplex-bin-avgonly


### Tuned variant (20s limit)
This solver ran with a 20-second time limit, offering a faster alternative to the baseline.


## Solver Comparison Table

| Run | Solver         | LP File                   | Bonds | Objective | Runtime (s) | Notes          |
|-----|----------------|---------------------------|-------|-----------|-------------|----------------|
| 1   | Baseline (MIP) | docplex-bin-avgonly.lp    | 21    | 40.2771   | 139.88      | python-mip CLP |


In [38]:
from pathlib import Path

CWD = Path.cwd()
print("CWD:", CWD)

lp_rel = Path("data/1/31bonds/docplex-bin-avgonly.lp")
lp_abs_guess = (CWD / lp_rel).resolve()
print("Expected LP path:", lp_abs_guess)
print("Exists?", lp_abs_guess.exists())


CWD: C:\Users\Admin\Documents\WISER_Optimization_VG
Expected LP path: C:\Users\Admin\Documents\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly.lp
Exists? True


In [39]:
from pathlib import Path

root = Path.cwd()
matches = list(root.rglob("docplex-bin-avgonly.lp"))
print("Matches found:", len(matches))
for i, p in enumerate(matches[:10], 1):
    print(f"{i}. {p}")

# Pick the first match if any were found
LP_ABS = matches[0] if matches else None
LP_ABS


Matches found: 1
1. C:\Users\Admin\Documents\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly.lp


WindowsPath('C:/Users/Admin/Documents/WISER_Optimization_VG/data/1/31bonds/docplex-bin-avgonly.lp')

In [40]:
from pathlib import Path

PROJECT_ROOT = Path.cwd()
LP_ABS = LP_ABS.resolve()
print("Found LP file at:", LP_ABS)

# Try to compute a clean relative path (if it’s inside the project)
try:
    LP_REL = str(LP_ABS.relative_to(PROJECT_ROOT))
except ValueError:
    # If it’s outside the project, just use the absolute path
    LP_REL = str(LP_ABS)

print("LP_REL set to:", LP_REL)


Found LP file at: C:\Users\Admin\Documents\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly.lp
LP_REL set to: data\1\31bonds\docplex-bin-avgonly.lp


In [41]:
from pathlib import Path

path_to_open = Path(LP_REL)
if not path_to_open.is_absolute():
    path_to_open = (Path.cwd() / path_to_open).resolve()

print("Opening:", path_to_open)
with open(path_to_open, "r", encoding="utf-8", errors="ignore") as f:
    head = "".join([next(f) for _ in range(5)])
print("First lines:\n", head)


Opening: C:\Users\Admin\Documents\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly.lp
First lines:
 \ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: VCIT _ VCIT (bench)

Minimize



In [42]:
LP_REL = str(lp_rel)
print("LP_REL set to:", LP_REL)


LP_REL set to: data\1\31bonds\docplex-bin-avgonly.lp


In [43]:
from pathlib import Path

path_to_open = Path(LP_REL)
if not path_to_open.is_absolute():
    path_to_open = (Path.cwd() / path_to_open).resolve()

print("Opening:", path_to_open)
with open(path_to_open, "r", encoding="utf-8", errors="ignore") as f:
    head = "".join([next(f) for _ in range(5)])
print("First lines:\n", head)


Opening: C:\Users\Admin\Documents\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly.lp
First lines:
 \ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: VCIT _ VCIT (bench)

Minimize



In [44]:
# Assuming solve_mip_tuned is already defined and imported in your notebook
x2, fx2, dt2 = solve_mip_tuned(LP_REL, max_seconds=20, mip_gap=0.0)

print("🔢 Bonds selected:", int(np.sum(x2)))
print("📈 Objective value:", round(fx2, 4))
print("⏱ Runtime (seconds):", round(dt2, 2))


🔢 Bonds selected: 19
📈 Objective value: 40.3562
⏱ Runtime (seconds): 151.54


In [3]:
import os
print(os.getcwd())


C:\Users\Admin\Documents\WISER_Optimization_VG\notebooks


In [4]:
with open("../data/1/31bonds/docplex-bin-avgonly.lp", "r") as f:
    lp_contents = f.read()
